## B - A Closer Look at Word Embeddings

> Ở các notebooks trước, chúng ta đã biết và làm việc với word embeddings (hay còn gọi là word vectors) khá nhiều. Trong notebook này, chúng ta sẽ cùng đào sâu hơn về word embeddings.

> Embeddings biến đổi một one-hot vector đã được mã hóa thành một vector số thực có dimension nhỏ hơn khá nhiều. One-hot vector còn được gọi là *sparse vector* - vector thưa, còn vector số thực được gọi là *dense vector*.

> Ý tưởng chính trong word embeddings là các từ có **bối cảnh - context** xuất hiện giống nhau thì sẽ **gần** nhau hơn trong vector space. Ví dụ: 2 từ nằm gần nhau trong vector space tức là khoảng cách Euclidean giữa chúng nhỏ. **Bối cảnh - context** ở đây được hiểu là **các từ đi kèm với từ mà ta đang quan tâm**. Ví dụ trong câu: "I purchased some items at the shop" và câu "I purchased some items at the store", từ "shop" và "store" xuất hiện trong cùng một bối cảnh nên chúng nằm gần nhau trong vector space.

> *word2vec* là một lớp các thuật toán tính toán word vectors từ corpus. Nếu muốn tìm hiểu thêm về *word2vec*, nhần vào [đây](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) và [đây](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/). Trong notebook này, chúng ta sẽ sử dụng *GloVe* vectors, *GloVe* là một thuật toán khác để tính word vectors. Nhấn vào [đây](https://nlp.stanford.edu/projects/glove/) để tìm hiểu thêm về *GloVe*.

> Trong PyTorch, ta sử dụng word vectors với `nn.embedding` layer. Layer này nhận đầu vào là tensor có size: **[sentence length, batch size]** và trả về tensor có size: **[sentence length, batch size, embedding dimensions]**

> Từ notebooks số 2 trở đi, ta đều sử dụng pre-trained word embeddings (cụ thể là GloVe vectors) do TorchText cung cấp. Embeddings này đã được huấn luyện trên kho dữ liệu khổng lồ. Ta có thể sử dụng pre-trained vectors này bên trong bất cứ model nào mà ta muốn. Ý tưởng khi áp dụng pre-trained word embedding là models sẽ học được context của mỗi từ, cho ta điểm xuất phát tốt hơn cho models của mình, dẫn đến việc training nhanh hơn và cho kết quả tốt hơn.

> Trong notebook này, chúng ta sẽ không training models nữa, thay vào đó ta sẽ nghiên cứu về word embeddings và đi tìm một vài sự thật thú vị về chúng.

> Phần lớn code trong này được tham khảo từ [đây](https://github.com/spro/practical-pytorch/blob/master/glove-word-vectors/glove-word-vectors.ipynb). Nếu bạn muốn đọc thêm về word embeddings, nhấn vào [đây](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/).

### Loading the GloVe vectors

> Đầu tiên, ta load GloVe vectors. Trường `name` xác định vectors nào được train, ở đây `6B` tức corpus có 6 tỷ words. Đối số `dim` xác định số chiều của word vectors. GloVe vector có số chiều nhận các giá trị: 50, 100, 200, 300. Bên cạnh đó còn có `42B` và `840B` vectors, tuy nhiên chúng mặc định có dimension là 300.

In [1]:
import torchtext.vocab

glove = torchtext.vocab.GloVe(name='6B', dim=100)

print('Loaded {} words'.format(len(glove.itos)))

Loaded 400000 words


> GloVe vocabulary có 400000 unique words. <br>
> **Trong tập GloVe vectors, tất cả các từ đều ở dạng lower-case**.

> `glove.vectors` thực chất laf1 tensor chứa các giá trị embeddings.

In [2]:
glove.vectors.shape

torch.Size([400000, 100])

> Chúng ta có thể kiểm tra các word liên kết với một hàng bằng cách kiểm tra `itos` (int to string) list.

In [3]:
glove.itos[0:10]

['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]

> Như ở trên, từ "the" liên quan tới vector hàng thứ 0, từ "," liên quan tới vector hàng thứ 1, etc.

> Ta cũng có thể làm ngược lại, sử dụng `stoi` (string to int) để tìm hàng liên quan tới từ. Lưu ý, nếu từ ta tìm kiếm không có trong vocabulary, torchtext sẽ thông báo lỗi.

In [4]:
glove.stoi["the"]

0

> Ta có thể lấy biểu diễn vector của một từ bằng cách sau:
>> 1. Đầu tiên lấy chỉ số của từ đó.
>> 2. Dùng chỉ số đó để truy xuất vào hàng tương ứng trong word embeddings.

In [5]:
index = glove.stoi["the"]
glove.vectors[index].shape

torch.Size([100])

> Ta sẽ viết 1 hàm thực hiện việc lấy embeddings của từ. Hàm này nhận vào từ và trả về vector liên quan, nếu không có từ trong vocabulary, nó sẽ trả về lỗi.

In [6]:
def get_vector(embeddings, word):
    assert word in embeddings.stoi, f'*{word}* not in vocabulary'
    index = embeddings.stoi[word]
    return embeddings.vectors[index]

In [7]:
get_vector(glove, "q").shape

torch.Size([100])

### Similar Contexts

> Bây giờ, chúng ta sẽ cùng nghiên cứu về context của các từ khác nhau.

> Nếu chúng ta muốn tìm các từ tương tự một từ cho trước, đầu tiên ta cần tìm biểu diễn vector của từ cho trước đó. Tiếp theo, ta quét khắp vocabulary để tính khoảng cách giữa từ trong từ điển và từ cho trước. Sau khi có được khoảng cách, ta sắp xếp theo thứ tự từ gần đến xa.

> Hàm dưới đây trả về 5 gần nhất đối với từ đầu vào.

In [8]:
import torch

def closest_words(embeddings, vector, n = 5):

    distance = [(word, torch.dist(vector, get_vector(embeddings, word))) for word in embeddings.itos]

    return sorted(distance, key = lambda w: w[1])[:n]

In [9]:
word_vector = get_vector(glove, "king")

closest_words(glove, word_vector)

[('king', tensor(0.)),
 ('prince', tensor(4.0922)),
 ('queen', tensor(4.2813)),
 ('monarch', tensor(4.4742)),
 ('brother', tensor(4.5367))]

> Tiếp theo, ta tạo hàm `print_tuples` để hiển thị tuples được trả về từ `closest_words`.

In [10]:
def print_tuples(tuples):
    for w, d in tuples:
        print(f'({d:02.04f}) {w}') 

In [12]:
word_vector = get_vector(glove, "king")
print_tuples(closest_words(glove, word_vector))

(0.0000) king
(4.0922) prince
(4.2813) queen
(4.4742) monarch
(4.5367) brother


### Anologies

> Một đặc tính khác của word embeddings là chúng có thể hoạt động giống các standard vector và cho ra kết quả thú vị.

In [13]:
def analogy(embeddings, word1, word2, word3, n=5):
    
    #get vectors for each word
    word1_vector = get_vector(embeddings, word1)
    word2_vector = get_vector(embeddings, word2)
    word3_vector = get_vector(embeddings, word3)
    
    #calculate analogy vector
    analogy_vector = word2_vector - word1_vector + word3_vector
    
    #find closest words to analogy vector
    candidate_words = closest_words(embeddings, analogy_vector, n+3)
    
    #filter out words already in analogy
    candidate_words = [(word, dist) for (word, dist) in candidate_words 
                       if word not in [word1, word2, word3]][:n]
    
    print(f'{word1} is to {word2} as {word3} is to...')
    
    return candidate_words

In [15]:
print_tuples(analogy(glove, 'man', 'king', 'woman'))


man is to king as woman is to...
(4.0811) queen
(4.6429) monarch
(4.9055) throne
(4.9216) elizabeth
(4.9811) prince


> Đây là một ví dụ kinh điển, cho thấy tính chất của word embeddings. Tại sao vector 'woman' + vector 'king' - vector 'man' = vector 'queen'?

> Ta có thể hiểu như sau: vector 'king' - vector 'man' = vector 'hoàng gia'. Vector 'hoàng gia' này khi liên kết với vector 'man' sẽ cho ra người đàn ông trong hoàng gia (trong trường hợp này chính là 'king'). Tương tự nếu ta cộng vector 'hoàng gia' này với vector 'woman', ta sẽ có vector liên kết tương đương là vector 'queen'. <br>
> Dưới đây là 1 ví dụ minh họa tương tự:

In [16]:
print_tuples(analogy(glove, 'man', 'actor', 'woman'))


man is to actor as woman is to...
(2.8133) actress
(5.0039) comedian
(5.1399) actresses
(5.2773) starred
(5.3085) screenwriter


In [17]:
print_tuples(analogy(glove, 'cat', 'kitten', 'dog'))


cat is to kitten as dog is to...
(3.8146) puppy
(4.2944) rottweiler
(4.5888) puppies
(4.6086) pooch
(4.6520) pug


In [18]:
print_tuples(analogy(glove, 'france', 'paris', 'england'))


france is to paris as england is to...
(4.1426) london
(4.4938) melbourne
(4.7087) sydney
(4.7630) perth
(4.7952) birmingham


In [19]:
print_tuples(analogy(glove, 'elvis', 'rock', 'eminem'))


elvis is to rock as eminem is to...
(5.6597) rap
(6.2057) rappers
(6.2161) rapper
(6.2444) punk
(6.2690) hop


In [20]:
print_tuples(analogy(glove, 'beer', 'barley', 'wine'))


beer is to barley as wine is to...
(5.6021) grape
(5.6760) beans
(5.8174) grapes
(5.9035) lentils
(5.9454) figs


### Correcting Spelling Mistakes

> Một trong những tính chất thú vị nữa của word embeddings là chúng có thể sử dụng trong việc sửa lỗi chính tả!

> Để tìm hiểu chi tiết hơn, click vào [đây](https://forums.fast.ai/t/nlp-any-libraries-dictionaries-out-there-for-fixing-common-spelling-errors/16411) và [đây](https://medium.com/m/global-identity?redirectUrl=https%3A%2F%2Fblog.heyday.xyz%2Fa-simple-spell-checker-built-from-word-vectors-9f28452b6f26).

> Đầu tiên, ta cần tập vocabulary lớn hơn của GloVe vectors, do lỗi chính tả không xuất hiện trong các vocabulary nhỏ.

**NOTE**: vectors có kích thước khá lớn (~2GB).

In [21]:
glove = torchtext.vocab.GloVe(name = '840B', dim = 300)

.vector_cache\glove.840B.300d.zip: 2.18GB [10:03, 3.61MB/s]                                
100%|█████████▉| 2196016/2196017 [05:47<00:00, 6313.76it/s]


> Kiểm tra kích thước vocabulary, ta thấy nó có hơn 2 triệu unique words.

In [22]:
glove.vectors.shape

torch.Size([2196017, 300])

> Do vectors được huấn luyện trên tập vocabulary rất lớn và trong corpus cũng rất lớn nên các từ xuất hiện có khá ít sự khác nhau. 

In [23]:
word_vector = get_vector(glove, 'korea')

print_tuples(closest_words(glove, word_vector))

(0.0000) korea
(3.9857) taiwan
(4.4022) korean
(4.9016) asia
(4.9593) japan


> Muốn sửa lỗi chính tả, đầu tiên ta cần tìm vector cho từ bị lỗi: 'reliable'.

In [24]:
word_vector = get_vector(glove, 'relieable')

print_tuples(closest_words(glove, word_vector))

(0.0000) relieable
(5.0366) relyable
(5.2610) realible
(5.4719) realiable
(5.5402) relable


> Chú ý, các viết đúng "reliable" không xuất hiện trong danh sách 10 từ gần nhất. Đáng nhẽ nó phải xuất hiện mới đúng?

> Giả thuyết đưa ra là: các từ sai chính tả thường xuất hiện trong các văn bản không quan trọng như:tweet, comment, etc. Trong khi các từ đúng chính tả thường xuất hiện trong các văn bản quan trọng, yêu cầu kiểm duyệt nghiêm ngặt. Từ đó, chúng không có cùng context nên không xuất hiện trong danh sách 10 từ gần nhất cũng là dễ hiểu.

> Thay vì chỉ sử dụng một example để tạo vector, ta sẽ sử dụng trung bình của nhiều examples và hy vọng kết quả sẽ tốt hơn.

In [25]:
reliable_vector = get_vector(glove, 'reliable')

reliable_misspellings = ['relieable', 'relyable', 'realible', 'realiable', 
                         'relable', 'relaible', 'reliabe', 'relaiable']

diff_reliable = [(reliable_vector - get_vector(glove, s)).unsqueeze(0) 
                 for s in reliable_misspellings]

>Trước tiên, tạo một vector đúng chính tả "reliable", sau đó tính độ sai khác giữa vector đúng chính tả và 8 vector sai chính tả. Tiếp theo ta ghép 8 "miss spelling tensors" này lại và unsqueeze batch dimension cho chúng.

> Tiếp đến, ta tính trung bình sai khác giữa 8 vector sai và vector đúng để tạo ra vector sai của riêng mình.

In [ ]:
misspelling_vector = torch.cat(diff_reliable, dim = 0).mean(dim = 0)


> Cuối cùng, ta sửa lỗi bằng cách tìm từ gần nhất đối với tổng của 'vector đã bị sai chính tả ban đầu và vector sai chính tả ta tự tạo ra'.

In [26]:
# vector sai chính tả ban đầu
word_vector = get_vector(glove, 'becuase')

# vector sai chính tả ta tự tạo ra
misspelling_vector = torch.cat(diff_reliable, dim = 0).mean(dim = 0)

print_tuples(closest_words(glove, word_vector + misspelling_vector))

(6.1090) because
(6.4250) even
(6.4358) fact
(6.4914) sure
(6.5094) though


In [27]:
word_vector = get_vector(glove, 'defintiely')

print_tuples(closest_words(glove, word_vector + misspelling_vector))

(5.4070) definitely
(5.5643) certainly
(5.7192) sure
(5.8152) well
(5.8588) always


In [28]:
word_vector = get_vector(glove, 'consistant')

print_tuples(closest_words(glove, word_vector + misspelling_vector))

(5.9641) consistent
(6.3674) reliable
(7.0195) consistant
(7.0299) consistently
(7.1605) accurate


In [29]:
word_vector = get_vector(glove, 'pakage')

print_tuples(closest_words(glove, word_vector + misspelling_vector))

(6.6117) package
(6.9315) packages
(7.0195) pakage
(7.0911) comes
(7.1241) provide


> Nếu muốn tìm hiểu sâu hơn về sửa lỗi chính tả, nhấn vào [đây](https://medium.com/m/global-identity?redirectUrl=https%3A%2F%2Fblog.heyday.xyz%2Fa-simple-spell-checker-built-from-word-vectors-9f28452b6f26).

## END